In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import os
import scipy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from scipy.integrate import cumtrapz
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
from sklearn.ensemble import GradientBoostingClassifier

import numpy as np 
import heapq
from joblib import load 

from xgboost import XGBClassifier



# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

# For interactive plots, uncomment the following line
# %matplotlib widget

In [3]:

all_train_data = []
for i in range(0, 50):
    all_train_data.append(Recording(f'data/train/train_trace_{i:03d}.pkl'))


In [25]:
import pandas as pd

data_list = []  # To store all the data points

'''# Process each recording
for i, recording in enumerate(all_train_data):
    # Ensure that each sensor has timestamps
    timestamps_ax = recording.data['ax'].timestamps
    timestamps_ay = recording.data['ay'].timestamps
    timestamps_az = recording.data['az'].timestamps
    timestamps_altitude = recording.data['altitude'].timestamps
    
    # Collect data for ax, ay, az, and altitude with actual timestamps
    data_list.extend([
        {"id": f"{i}_ax", "time": ts, "sensor": "ax", "value": val}
        for ts, val in zip(timestamps_ax, recording.data['ax'].values)
    ])
    data_list.extend([
        {"id": f"{i}_ay", "time": ts, "sensor": "ay", "value": val}
        for ts, val in zip(timestamps_ay, recording.data['ay'].values)
    ])
    data_list.extend([
        {"id": f"{i}_az", "time": ts, "sensor": "az", "value": val}
        for ts, val in zip(timestamps_az, recording.data['az'].values)
    ])
    data_list.extend([
        {"id": f"{i}_altitude", "time": ts, "sensor": "altitude", "value": val}
        for ts, val in zip(timestamps_altitude, recording.data['altitude'].values)
    ])

# Convert to DataFrame
df = pd.DataFrame(data_list)'''


# Process each recording
for i, recording in enumerate(all_train_data):

    timestamps_altitude = recording.data['altitude'].timestamps
    
    data_list.extend([
        {"id": i, "time": ts, "value": val}
        for ts, val in zip(timestamps_altitude, recording.data['altitude'].values)
    ])

# Convert to DataFrame
df = pd.DataFrame(data_list)
df.head(20)
    


,id,time,value
0,0,0.000000,407.332413
1,0,0.079890,407.332413
2,0,0.159780,407.332413
3,0,0.239670,407.332413
4,0,0.319560,407.332413
5,0,0.399450,407.332413
6,0,0.479339,407.332413
7,0,0.559229,407.332413
8,0,0.639119,407.332413
9,0,0.719009,407.332413


In [17]:
import tsfresh
from tsfresh.examples import load_robot_execution_failures
tsfresh.examples.robot_execution_failures.download_robot_execution_failures()
df_ex, _ = load_robot_execution_failures()
df_ex.head(10)

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0
5,1,5,-1,-1,63,-3,-1,0
6,1,6,-1,-1,63,-3,0,0
7,1,7,-1,-1,63,-3,-1,0
8,1,8,-1,-1,63,-3,-1,0
9,1,9,-1,-1,61,-3,0,0


In [21]:
# Print all unique IDs to check if they are being generated correctly
unique_ids = df['id'].unique()
print("Unique IDs found:", unique_ids)


Unique IDs found: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49']


In [26]:
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute

# Extract features
extracted_features = extract_features(df, column_id='id', column_sort='time', impute_function=impute)


Feature Extraction:   0%|          | 0/17 [00:00<?, ?it/s]

In [8]:
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.selection import select_features
from tsfresh import extract_relevant_features





X = extracted_features


y = []
for recording in all_train_data:
    y.append(recording.labels['path_idx'])
y = np.array(y) 

X_filtered = select_features(X, y)

features_filtered_direct = extract_relevant_features(X_filtered, y, column_id='id', column_sort='time')

X_train, X_test, y_train, y_test = train_test_split(X_filtered, y, test_size=0.2, random_state=42)



ValueError: df must be a DataFrame or a dict of DataFrames. See https://tsfresh.readthedocs.io/en/latest/text/data_formats.html

In [ ]:
clf = RandomForestClassifier(random_state=42)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [5, 10],
    'max_depth': [ 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform a grid search with cross validation and print training and test scores
grid_search = GridSearchCV(clf, param_grid, cv=2, n_jobs=-1, return_train_score=True)
grid_search.fit(X_train, y_train)

results = grid_search.cv_results_

# Print the mean training and testing scores for each parameter set
for i in range(len(results['params'])):
    print(f"Parameters: {results['params'][i]}")
    print(f"Mean Training Score: {results['mean_train_score'][i]:.3f}")
    print(f"Mean Test Score: {results['mean_test_score'][i]:.3f}")
    print('-' * 40)

# If you want to print scores for each individual fold as well
for i in range(len(results['params'])):
    for j in range(2):
        print(f"Parameters: {results['params'][i]} - Fold {j+1}")
        print(f"Training Score: {results[f'split{j}_train_score'][i]:.3f}")
        print(f"Test Score: {results[f'split{j}_test_score'][i]:.3f}")
    print('-' * 40)


# Print the best parameters
print(f"Best parameters: {grid_search.best_params_}")

# Train the classifier with the best parameters
clf = RandomForestClassifier(**grid_search.best_params_, random_state=42)
clf.fit(X_train, y_train)


# Predict the labels of the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate balanced accuracy
balanced_accuracy = sklearn.metrics.balanced_accuracy_score(y_test, y_pred, adjusted=True)
print(f"Balanced Accuracy: {balanced_accuracy}")


# Plot the confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')


# plot y_test2 and y_pred2

plt.figure()
plt.plot(y_test, label='y_test2')
plt.plot(y_pred, label='y_pred2')
plt.legend()
plt.show()